In [1]:
import sys
sys.path.append('/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/')
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import yfinance as yf
import time
from datetime import timedelta, datetime
from systems.performance_reporter import StrategyReporter, PerformanceReporter
from systems.utils import MarketDataExtractor

In [2]:
market_data_extractor = MarketDataExtractor()
strategy_reporter = StrategyReporter()
performance_reporter = PerformanceReporter(market_data_extractor)

In [3]:
# All the Inputs for the Backtest that needs to be analyzed
testname = '4c4f66a4e5697c5342a7e4dee1e145be8a85d133a7300ee6505a13895989a8e7' # Profits not Reinvested
# testname = '3aa6bc1bc58ab86f2a23419d3737ec62991783e80d8d8802f5cc68a1cfb0a33b' # Profits Reinvested & Margin Deployed @ 0.65 of 3.3x
save_folder = "/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/data/yahoo/1d"
test_folder_path = f'/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/vault/backtest_reports/{testname}'
backtest_output = strategy_reporter.load_backtest_output(test_folder_path)
open_orders = backtest_output['open_orders']
closed_orders = backtest_output['closed_orders']
config_dict = backtest_output['config_dict']

In [4]:
#Processing Open order to make them close, and remove history
open_orders_list = strategy_reporter.clean_up_order_list(open_orders, drop_history=True)
#Processing Closed orders to remove history
closed_orders_list = strategy_reporter.clean_up_order_list(closed_orders, drop_history=True)
#Merge both orders
final_list = closed_orders_list + open_orders_list
# Step 2: Sort the filtered orders by exit order `filled_timestamp`
sorted_orders = sorted(final_list, key=lambda x: x[1]['filled_timestamp'])
# Step 3: Fetch the first and last `filled_timestamp` with buffer
start_date = sorted_orders[0][1]['filled_timestamp'] - timedelta(days=365)
end_date = sorted_orders[-1][1]['filled_timestamp'] + timedelta(days=365)
# end_date = pd.Timestamp(datetime(2010, 1, 1))
# Current we are downloading csv file for below three index
indices = ["^DJI", "^IXIC", "^GSPC"]

index_data_dict = strategy_reporter.load_and_save_index_data(indices, start_date, end_date, save_folder)
rolling_window = int(max(60, (end_date - start_date).days / 20))
print({f"Analyzing with a rolling window of {rolling_window} days"})

{'Analyzing with a rolling window of 378 days'}


### This below function calculates profit/loss from the orders  **(exitPrice - entryPrice) * orderQuantity**

This function also plot the progress lines for 3 indexes provided earlier.

In [5]:
starting_capital = 72181
trade_df_by_date = strategy_reporter.create_data_input_for_cumulative_returns_and_indices(final_list, index_data_dict, starting_capital)
strategy_reporter.plot_cumulative_returns_and_indices(trade_df_by_date, index_data_dict)

# Plot Rolling charts

#### Sharpe Ratio

In [6]:
index_data_dict_ = index_data_dict.copy()
trade_df_by_date_ = trade_df_by_date.copy()
rolling_sharpe_df = strategy_reporter.calculate_rolling_sharpe_ratio(trade_df_by_date_, index_data_dict_, index_name='^IXIC', window=rolling_window)
# rolling_sharpe_df.plot(title='Rolling Sharpe Ratio', figsize=(35, 5), grid=True)
strategy_reporter.plot_rolling_sharpe_ratio(rolling_sharpe_df, index_data_dict)

### Rolling Win Percentage

In [7]:
trade_df_by_order = strategy_reporter.prepare_trade_data_df_by_order(final_list)
strategy_reporter.plot_rolling_win_percentage(trade_df_by_order, index_data_dict, window=rolling_window)

### Drawdown Analysis

In [8]:
trade_df_by_date_drawdown = strategy_reporter.calculate_drawdowns(trade_df_by_date)
strategy_reporter.plot_drawdown(trade_df_by_date_drawdown, index_data_dict)

### Plot Trades of a Single Asset

In [1]:
# Add order_num to the final_list
for order_num, order in enumerate(final_list):
    for i in range(len(order)):
        order[i]['order_num'] = order_num

# Get list of all the symbols in final_list
all_symbols = list(set([x[1]['symbol'] for x in final_list]))
# from pprint import pprint
# pprint(str(all_symbols))

NameError: name 'final_list' is not defined

In [10]:
# Add order_num to the final_list
for order_num, order in enumerate(open_orders_list):
    for i in range(len(order)):
        order[i]['order_num'] = order_num

# Get list of all the symbols in final_list
open_symbols = list(set([x[1]['symbol'] for x in open_orders_list]))
from pprint import pprint
pprint(str(open_symbols))

"['CVNA', 'MCHP', 'VST', 'MSTR', 'VALE', 'APP', 'PLTR', 'BSBR', 'BA']"


In [11]:
import random
symbol = 'ADBE' #random.choice(symbols)
if symbol in all_symbols:
    strategy_reporter.process_and_plot_orders(final_list, symbol, save_folder)
else:
    print(f"Symbol {symbol} not found in the final list of orders")